##Problem Statement

Suppose you're trying to help a company determine which computers to purchase. The company has been able to
pull utilization data by employee that classifies users into 3 bins, depending on how much they use their computer in
their work:

* Low usage - spends a lot of time in meetings, checking email, doing people management
* Average usage - requires some compute power, with balanced mix of heads down/technical work along with a
good amount of meetings/email writing
*  High usage - power user, relies heavily on computer performance

Additionally, they've surveyed employees to collect the relative importance of the following variables describing a
computer's performance:
* Memory
* Processing
* Storage
* Price inverse - this metric was given to you by the company as you can see in the dataset, with the directive that
price inverse being fixed at a 25% weight in the purchase decision.

The results of the survey data can be found here.

Lastly, the company is looking to purchase a maximum of **3** different computer models, and have compiled the following
list scoring their **memory, processing, storage, and relative price**. Each dimension is scored from **0-10**, with 10 being the
best.

**Given this information, provide the company with a recommendation on which computers to purchase.**

In [1]:
import requests
import os
import pandas as pd
import numpy as np

Reading data to the dataframe

In [2]:
url = "https://raw.githubusercontent.com/shubhamkalra27/dsep-2020/main/datasets/util_b_emp.csv"
utilization_data = pd.read_csv(url)
utilization_data

,employee_id,utilization_bin
0,1743,high
1,1752,high
2,1758,high
3,1825,high
4,1842,high
...,...,...
141,1463,low
142,1625,low
143,1377,low
144,1353,low


In [3]:
url2 = "https://raw.githubusercontent.com/shubhamkalra27/dsep-2020/main/datasets/survey_emp.csv"
survey_data = pd.read_csv(url2)
survey_data

,employee_id,memory,processing,storage,inverse_price
0,1743,0.375,0.225,0.150,0.25
1,1752,0.450,0.225,0.075,0.25
2,1758,0.375,0.300,0.075,0.25
3,1825,0.300,0.300,0.150,0.25
4,1842,0.300,0.300,0.150,0.25
...,...,...,...,...,...
141,1463,0.225,0.150,0.375,0.25
142,1625,0.225,0.150,0.375,0.25
143,1377,0.150,0.225,0.375,0.25
144,1353,0.150,0.225,0.375,0.25


In [4]:
url3 = "https://raw.githubusercontent.com/shubhamkalra27/dsep-2020/main/datasets/vendor_options.csv"
vendor_options = pd.read_csv(url3)
vendor_options

,computer_id,memory,processing,storage,inverse_price
0,13,5,7,10,2.7
1,16,9,8,9,1.3
2,4,8,9,10,1.0
3,1,8,8,9,1.7
4,3,5,4,4,5.7
5,2,6,7,7,3.3
6,20,7,10,7,2.0
7,8,9,6,9,2.0
8,9,9,8,7,2.0
9,7,7,7,9,2.3


Before we proceed to extracting information and interpreting, we should perform data cleaning. The aim of this step is to obtain quality, integrated and consistent data to perform accurate analysis.

###Data Cleaning

It would be accurate to assure that there are no duplicates. Let's get rid of them if there are any.

In [5]:
utilization_data = utilization_data.drop_duplicates()
survey_data = survey_data.drop_duplicates()
vendor_options = vendor_options.drop_duplicates()
print(utilization_data.shape, survey_data.shape, vendor_options.shape)

(146, 2) (146, 5) (11, 5)


As we can see based on the shapes, there were no duplicates, there are no two identical employee_id(which is the most important in case of those dataframes), and no two identical computer_id. Considering the vendor_options dataframe, let's notice that if two computers have all the same values of all features apart from the id, they are actually identical for us(as we have no further information).

We should check if there are no identical computers, by means of our point of view.

In [6]:
vendor_options = vendor_options.drop_duplicates(subset=['memory', 'processing', 'storage', 'inverse_price'], keep='first')
vendor_options

,computer_id,memory,processing,storage,inverse_price
0,13,5,7,10,2.7
1,16,9,8,9,1.3
2,4,8,9,10,1.0
3,1,8,8,9,1.7
4,3,5,4,4,5.7
5,2,6,7,7,3.3
6,20,7,10,7,2.0
7,8,9,6,9,2.0
8,9,9,8,7,2.0
9,7,7,7,9,2.3


Now, let's chceck if there are some empty values.

In [7]:
print("Utilization_data: ", utilization_data.isnull().values.any(), " Survey_data: ", survey_data.isnull().values.any(), 
      " Vendor_options: ", vendor_options.isnull().values.any())

Utilization_data:  False  Survey_data:  False  Vendor_options:  False


Luckily, there are no empty values so there is no need to figuring out how can we handle them.

Let's check the data types in our dataframes.

In [8]:
print(utilization_data.dtypes)
print(survey_data.dtypes)
print(vendor_options.dtypes)

employee_id         int64
utilization_bin    object
dtype: object
employee_id        int64
memory           float64
processing       float64
storage          float64
inverse_price    float64
dtype: object
computer_id        int64
memory             int64
processing         int64
storage            int64
inverse_price    float64
dtype: object


###Approach

The approach that will be used to solve the problem is simple and clearly mathematical. Taking the quantity of data we can use, performing Machine Learning or Deep Learning methods is not a good idea. 

We will check how each of the features among *memory*,	*processing*, and	*storage* is important for particular level of utilization. This will result is obtaining the weigths, which we want to use to calculate weighted sum for each computer. 
This approach ensures that computers for all values of *utilization_bin* are selected.

###Extracting information

Our data is clean, so we can start performing calculations in order to extract the information.

Joining *utilization_data* to *survey_data* according to the *employee_id* will be a good first step. The resulting DataFrame will allow us to check what are the values of *memory*,	*processing*, and *storage* that determine the level of *utilization_bin*.

In [9]:
util_survey = pd.merge(utilization_data,survey_data, on="employee_id")
util_survey

,employee_id,utilization_bin,memory,processing,storage,inverse_price
0,1743,high,0.375,0.225,0.150,0.25
1,1752,high,0.450,0.225,0.075,0.25
2,1758,high,0.375,0.300,0.075,0.25
3,1825,high,0.300,0.300,0.150,0.25
4,1842,high,0.300,0.300,0.150,0.25
...,...,...,...,...,...,...
141,1463,low,0.225,0.150,0.375,0.25
142,1625,low,0.225,0.150,0.375,0.25
143,1377,low,0.150,0.225,0.375,0.25
144,1353,low,0.150,0.225,0.375,0.25


Let's check if our new dataframe cointains information for all utilization levels described in the problem statement.

In [10]:
high_count = util_survey['utilization_bin'].value_counts()["high"]
average_count = util_survey['utilization_bin'].value_counts()["medium"]
low_count = util_survey['utilization_bin'].value_counts()["low"]
print(high_count, average_count, low_count)

40 86 20


Let's calculate the mean values of the features for each level of usage:

In [11]:
average_weights = util_survey.groupby('utilization_bin')['memory', 'processing', 'storage', 'inverse_price'].mean()
average_weights

<ipython-input-11-c93cae1cfd94>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_weights = util_survey.groupby('utilization_bin')['memory', 'processing', 'storage', 'inverse_price'].mean()


,memory,processing,storage,inverse_price
utilization_bin,,,,
high,0.378750,0.260625,0.110625,0.25
low,0.195000,0.150000,0.405000,0.25
medium,0.185756,0.339244,0.225000,0.25


In [12]:
average_weights.dtypes

memory           float64
processing       float64
storage          float64
inverse_price    float64
dtype: object

Our weights should sum up to 1. Let's check if this actuaally happens, and if not, we have to choose another degree of rounding our float64 values.

In [13]:
average_weights.sum(axis=1)

utilization_bin
high      1.0
low       1.0
medium    1.0
dtype: float64

They are summing up to 1. Now we can state oour interpretation, that values in *average_weights* are the percentages, how much we want the values of the features in our calculations to be important.

A we have the weights of *memory*, * processing *, *storage * , and *inverse_price* calculated for each *utilization_bin* value, we can now calculate the score of each computer.

Let's choose the best computer for low usage, by calculating the weighted sum of our features for each computer and extract the maximum value.

In [14]:
columns = ['memory', 'processing', 'storage', 'inverse_price']

In [15]:
score_for_low = (average_weights.loc['low'] * vendor_options[columns])
vendor_options['score_for_low'] = score_for_low.sum(axis=1)

score_for_average = (average_weights.loc['medium'] * vendor_options[columns])
vendor_options['score_for_medium'] = score_for_average.sum(axis=1)

score_for_high = (average_weights.loc['high'] * vendor_options[columns])
vendor_options['score_for_high'] = score_for_high.sum(axis=1)
vendor_options

,computer_id,memory,processing,storage,inverse_price,score_for_low,score_for_medium,score_for_high
0,13,5,7,10,2.7,6.750,6.228488,5.499375
1,16,9,8,9,1.3,6.925,6.735756,6.814375
2,4,8,9,10,1.0,7.210,7.039244,6.731875
3,1,8,8,9,1.7,6.830,6.650000,6.535625
4,3,5,4,4,5.7,4.620,4.610756,4.803750
5,2,6,7,7,3.3,5.880,5.889244,5.696250
6,20,7,10,7,2.0,6.200,6.767733,6.531875
7,8,9,6,9,2.0,6.800,6.232267,6.468125
8,9,9,8,7,2.0,6.290,6.460756,6.768125
9,7,7,7,9,2.3,6.635,6.275000,6.046250


In [16]:
max_values = vendor_options[['score_for_low', 'score_for_medium', 'score_for_high']].max()

In [17]:
max_low_score_idx = vendor_options['score_for_low'].idxmax()
choice_for_low = vendor_options.loc[max_low_score_idx, 'computer_id']

max_medium_score_idx = vendor_options['score_for_medium'].idxmax()
choice_for_medium = vendor_options.loc[max_medium_score_idx, 'computer_id']

max_high_score_idx = vendor_options['score_for_high'].idxmax()
choice_for_high = vendor_options.loc[max_high_score_idx, 'computer_id']


print("The ID of perfect computer for low usage: ", choice_for_low)
print("The ID of perfect computer for average usage: ", choice_for_medium)
print("The ID of perfect computer for high usage: ", choice_for_high)


The ID of perfect computer for low usage:  4
The ID of perfect computer for average usage:  4
The ID of perfect computer for high usage:  16


We can see that low and average bins got the same computer. As we want to buy maximum 3 of them we can just buy computer with ID 4 for both low and average usage, or choose other one for one of the bins. Let's get through the average weights to analyze importances of features.

In [18]:
average_weights

,memory,processing,storage,inverse_price
utilization_bin,,,,
high,0.378750,0.260625,0.110625,0.25
low,0.195000,0.150000,0.405000,0.25
medium,0.185756,0.339244,0.225000,0.25


Storage is the most important feature for low usage, and processing is vital for medium usage. Taking into consideration that computer of ID 4 has the highest possible score for storage, and lower value for processing, we will choose the other computer, preferably with the highest processing value.

In [22]:
max_processing_idx = vendor_options['processing'].idxmax()
other_medium_choice = vendor_options.loc[max_processing_idx, 'computer_id']
other_medium_choice

20

Let's check how good according to the rank is computer with the ID equal to 20.

In [24]:
df_sorted = vendor_options.sort_values('score_for_medium', ascending = False)
df_sorted

,computer_id,memory,processing,storage,inverse_price,score_for_low,score_for_medium,score_for_high
2,4,8,9,10,1.0,7.210,7.039244,6.731875
6,20,7,10,7,2.0,6.200,6.767733,6.531875
1,16,9,8,9,1.3,6.925,6.735756,6.814375
3,1,8,8,9,1.7,6.830,6.650000,6.535625
8,9,9,8,7,2.0,6.290,6.460756,6.768125
9,7,7,7,9,2.3,6.635,6.275000,6.046250
7,8,9,6,9,2.0,6.800,6.232267,6.468125
0,13,5,7,10,2.7,6.750,6.228488,5.499375
5,2,6,7,7,3.3,5.880,5.889244,5.696250
4,3,5,4,4,5.7,4.620,4.610756,4.803750


W can see, that the computer with the best processing score is the second best in the rank. Therefore, it looks like the computer of the ID equal to 20 is the good choice for the medium usage.

The final results with the updated medium choice:

In [25]:
choice_for_medium = other_medium_choice
print("The ID of perfect computer for low usage: ", choice_for_low)
print("The ID of perfect computer for average usage: ", choice_for_medium)
print("The ID of perfect computer for high usage: ", choice_for_high)

The ID of perfect computer for low usage:  4
The ID of perfect computer for average usage:  20
The ID of perfect computer for high usage:  16


In [27]:
final_choices = {'low': choice_for_low, 'medium': choice_for_medium, 'high': choice_for_high}
final_choices

{'low': 4, 'medium': 20, 'high': 16}